# Analyze Results

In [ ]:
import os.path
import numpy as np
import pandas as pd

from hit_prediction_code.analytics import get_results_as_dataframe
from hit_prediction_code import analytics

results = get_results_as_dataframe(
    project_name='hit-prediction-ismir2020',
    table_name='hit_prediction',
    filter_git_dirty=True,
#     date_filter='BETWEEN \'2021-04-07 14:00:00\' AND \'2021-04-13 17:00:00\'',
    date_filter='>= \'2021-04-07 14:00:00\'',
    columns=['id', 'sourcefile', 'outcome'],
    filters=['sourcefile LIKE \'plans/wide_and_deep_performance/%%ordinal_one_hot_all_lfmlc.py\''],
#     filters=['sourcefile LIKE \'plans/ordinal%%\''],
)

analytics.add_approach_to_df(results)
analytics.add_cv_epoch_evaluator_outcome_to_df(results)

results.sort_values(by='sourcefile', inplace=True)
display(len(results))

In [ ]:
normalize = 'true'
# normalize = False

for _, result in results.iterrows():
    mean_result = result['mean']
    display(mean_result)

    normalized_cm = []
    for cm in mean_result.loc['confusion_matrix']:
        if normalize:
            cm = analytics.normalize_confusion_matrix(cm, method=normalize)
        else:
            cm = np.array(cm)
        normalized_cm.append(cm)
    display(result['approach'])
    outcome = analytics.scores_from_confusion_matrices(normalized_cm, mean_result.columns)
    display(outcome)
    best_f1 = outcome[outcome['macro_f1'] >= outcome['macro_f1'].max()]
    display(best_f1)
    
    plot_shape = (1, 1)
    fig_width = 21
    if len(normalized_cm) >= plot_shape[0] * plot_shape[1]:
        fig_size = (fig_width, int(fig_width / plot_shape[1] * plot_shape[0] * 0.85))
        analytics.plot_epochs_confution_matrix(result['approach'], normalized_cm, mean_result.columns, plot_shape=plot_shape, figsize=fig_size)
        
    analytics.plot_reg(np.array(mean_result.loc['confusion_matrix'][best_f1.index[0]]))